In [1]:
# add additional processes to serve as workers 
n_procs = 1
addprocs(n_procs);  

In [2]:
# add present working directory to path from which to load modules
push!(LOAD_PATH, ".") 

# load module DynamicProgramming on each of the processes 
using DynamicProgramming
using Interpolations
for p in workers()
    remotecall_fetch(p, eval, :(using DynamicProgramming))
    remotecall_fetch(p, eval, :(using Interpolations))
end

In [3]:
## define the dynamic programming problem 

# dynamics 
@everywhere function f(t::Int64, x::Array{Float64, 1}, u::Array{Float64, 1}, theta::Array{Float64, 1}) 
    xu = x[1]*(1-u[1])
    # sensitivity w.r.t. r 
    Df_r = xu*(1 - x[1]*(1-u[1]))
    Df_xt = (1+theta[1])*(1 - u[1]) - 2*theta[1]*(1 - u[1])^2*x[1]  
    return [xu + theta[1]*xu*(1 - xu); Df_r + Df_xt*x[2]]
end

# nominal parameter value 
@everywhere theta0 = [0.5, 1000.0]
f(0, [1.; 0.], [0.3], theta0)

# time horizon 
@everywhere N = 25

# state dimension 
@everywhere n = 2

# stage reward function 
@everywhere function phi(x::Array{Float64, 1}, u::Array{Float64, 1}, theta::Array{Float64, 1})
    if x[1] == 0 
        return 0
    else
        return  x[1]*(1 - u[1]) + (theta[2]/x[1])^2*(1 - u[1])*x[2]^2 
    end
end

# define state grid 
@everywhere nx1 = 50 # number of grid points in each dimension  
@everywhere nx2 = 50
@everywhere xgrid = (linspace(0, 1, nx1), linspace(-1, 1, nx2)) 

# define input grid 
nu1 = 100
@everywhere ugrid = (linspace(0, 1, nx1),)
length(ugrid)

1

In [4]:
# compute value function and optimal policy 
J = dp_loop(f, phi, ugrid, xgrid, theta0, N)

# compute optimal trajectory from initial condition x0 
# x0 = [1.; 0.]
# (x_opt, u_opt) = dp_forward(J, u, f, x0, xgrid, theta0, N); 
# xgrid[2][1]

50x50x25 SharedArray{Float64,3}:
[:, :, 1] =
 0.0         0.0         0.0         …  0.0         0.0         0.0       
 2.80563e11  2.67209e11  2.53863e11     3.43923e11  3.58564e11  3.73213e11
 1.17539e11  1.11054e11  1.04586e11     1.79661e11  1.86866e11  1.94079e11
 6.55156e10  6.14405e10  5.7387e10      1.25892e11  1.3063e11   1.35373e11
 4.13131e10  3.85456e10  3.57782e10     9.94056e10  1.02918e11  1.0643e11 
 2.8234e10   2.62006e10  2.43084e10  …  8.37243e10  8.65017e10  8.92917e10
 2.0423e10   1.91003e10  1.77777e10     7.3336e10   7.56303e10  7.79246e10
 1.58776e10  1.4801e10   1.3981e10      6.60226e10  6.79676e10  6.99288e10
 1.30587e10  1.23147e10  1.15707e10     6.05222e10  6.22249e10  6.39276e10
 1.12851e10  1.07883e10  1.04006e10     5.60486e10  5.75387e10  5.90288e10
 1.04729e10  1.02857e10  1.00984e10  …  5.29621e10  5.43053e10  5.56485e10
 9.96964e9   9.93891e9   9.90818e9      4.98758e10  5.10706e10  5.22654e10
 9.57876e9   9.72474e9   9.90599e9      4.79447e10  4.9

In [5]:
using Gadfly
using Colors

berkeley_blue = RGB((1/256*[ 45.,  99., 127.])...)
berkeley_gold = RGB((1/256*[224., 158.,  25.])...); 

LoadError: LoadError: ArgumentError: Gadfly not found in path
while loading In[5], in expression starting on line 1

In [6]:
# plot optimal state trajectory 
p = plot(x=1:N, y=theta0[2]*x_opt[1, :], 
            Guide.XLabel("Time (t)"), 
            Guide.YLabel("Population (x<sub>t</sub> )"), 
            Geom.line, Geom.point,
            Theme(default_color=berkeley_blue)
)
draw(PDF("flies_optimal_input.pdf", 15cm, 10cm), p)
p

LoadError: LoadError: UndefVarError: x_opt not defined
while loading In[6], in expression starting on line 2

In [7]:
# plot optimal input sequence 
p = plot(x=1:N-1, y=u_opt, 
        Guide.XLabel("Time (t)"), 
        Guide.YLabel("Input (u<sub>t</sub> )"), 
        Geom.line, Geom.point,
        Theme(default_color=berkeley_gold)
)
draw(PDF("flies_optimal_states.pdf", 15cm, 10cm), p)
p

LoadError: LoadError: UndefVarError: u_opt not defined
while loading In[7], in expression starting on line 2

In [8]:
using Debug